In [1]:
from pennylane import FermiC, FermiA

a0_dag = FermiC(0)
a1 = FermiA(1)

In [2]:
fermi_word = a0_dag * a1
fermi_sentence = 1.3 * a0_dag * a1 + 2.4 * a1
fermi_sentence

1.3 * a⁺(0) a(1)
+ 2.4 * a(1)

In [3]:
fermi_sentence = fermi_sentence * fermi_word + 2.3 * fermi_word
fermi_sentence

1.3 * a⁺(0) a(1) a⁺(0) a(1)
+ 2.4 * a(1) a⁺(0) a(1)
+ 2.3 * a⁺(0) a(1)

In [4]:
fermi_sentence = 1.2 * a0_dag + 0.5 * a1 - 2.3 * (a0_dag * a1) ** 2
fermi_sentence

1.2 * a⁺(0)
+ 0.5 * a(1)
+ -2.3 * a⁺(0) a(1) a⁺(0) a(1)

In [5]:
from pennylane import jordan_wigner

pauli_sentence = jordan_wigner(fermi_sentence)
pauli_sentence

(0.6*(PauliX(wires=[0]))) + (-0.6j*(PauliY(wires=[0]))) + (0.25*(PauliZ(wires=[0]) @ PauliX(wires=[1]))) + (0.25j*(PauliZ(wires=[0]) @ PauliY(wires=[1]))) + (0j*(Identity(wires=[0]))) + (0j*(PauliZ(wires=[1]))) + (0j*(PauliZ(wires=[0]))) + (0j*(PauliZ(wires=[0]) @ PauliZ(wires=[1])))

In [6]:
h1 = 0.01 * (FermiC(0) * FermiA(0) + FermiC(1) * FermiA(1))
h2 = -0.02 * (FermiC(0) * FermiA(1) + FermiC(1) * FermiA(0))
h = h1 + h2
print(h)

0.01 * a⁺(0) a(0)
+ 0.01 * a⁺(1) a(1)
+ -0.02 * a⁺(0) a(1)
+ -0.02 * a⁺(1) a(0)


In [7]:
h = jordan_wigner(h)

In [8]:
from pennylane import numpy as np

val, vec = np.linalg.eigh(h.sparse_matrix().toarray())
print(f"eigenvalues:\n{val}")
print()
print(f"eigenvectors:\n{np.real(vec.T)}")

eigenvalues:
[-0.01  0.    0.02  0.03]

eigenvectors:
[[-0.         -0.70710678 -0.70710678 -0.        ]
 [ 1.          0.          0.          0.        ]
 [ 0.          0.          0.          1.        ]
 [ 0.         -0.70710678  0.70710678  0.        ]]


In [9]:
import pennylane as qml

symbols = ["H", "H"]
geometry = np.array([[-0.67294, 0.0, 0.0], [0.67294, 0.0, 0.0]], requires_grad=False)

In [10]:
mol = qml.qchem.Molecule(symbols, geometry)
core, one, two = qml.qchem.electron_integrals(mol)()

In [11]:
for i in range(4):
    if i < 2:
        one = one.repeat(2, axis=i)
    two = two.repeat(2, axis=i)

In [12]:
import itertools

n = one.shape[0]

h = 0.0

for p, q in itertools.product(range(n), repeat=2):
    if p % 2 == q % 2:  # to account for spin-forbidden terms
        h += one[p, q] * FermiC(p) * FermiA(q)